# Colab FastAPI + Ollama + YOLO

"
            "This notebook provisions the FastAPI project, installs Ollama/YOLO, and exposes the API via ngrok.
"
            "Run the cells top-to-bottom whenever you need a fresh Colab runtime.

## Before you run
"
            "1. Upload this repository (or sync from Git) into `/content/colab_fastapi`.
"
            "2. (Optional) Add your ngrok authtoken to the Colab secrets manager and set `NGROK_AUTHTOKEN`.
"
            "3. Decide which Ollama + YOLO models you want to use; defaults are `phi3` and `yolov8n.pt`.

In [33]:
from pathlib import Path
import os

PROJECT_ROOT = Path('/content/colab_fastapi').resolve()
if not PROJECT_ROOT.exists():
    raise RuntimeError('Upload the project into /content/colab_fastapi before continuing.')

os.chdir(PROJECT_ROOT)
print('Working directory:', PROJECT_ROOT)

Working directory: /content/colab_fastapi


In [34]:
import os

os.environ.setdefault('FASTAPI_PORT', '8000')
os.environ.setdefault('OLLAMA_MODEL', 'phi3')
os.environ.setdefault('YOLO_MODEL', 'yolov8n.pt')
os.environ.setdefault('YOLO_CONFIDENCE', '0.35')
os.environ.setdefault('NGROK_AUTHTOKEN', '2dX3AiyaMZ9bc5JFBUPre9SRd0L_kKit9xXv5s7hf7E8k8Ei')

print('FASTAPI_PORT =', os.environ['FASTAPI_PORT'])
print('OLLAMA_MODEL =', os.environ['OLLAMA_MODEL'])
print('YOLO_MODEL =', os.environ['YOLO_MODEL'])
print('YOLO_CONFIDENCE =', os.environ['YOLO_CONFIDENCE'])
print('NGROK_AUTHTOKEN =', os.environ.get('NGROK_AUTHTOKEN', '<unset>'))

FASTAPI_PORT = 8000
OLLAMA_MODEL = phi3
YOLO_MODEL = yolov8n.pt
YOLO_CONFIDENCE = 0.35
NGROK_AUTHTOKEN = <unset>


In [35]:
%%bash
cd /content/colab_fastapi
sudo apt-get update -y
sudo apt-get install -y curl git
pip install --upgrade pip
pip install -r requirements.txt

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.81.0-1ubuntu1.21).
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded,

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [36]:
%%bash
cd /content/colab_fastapi
chmod +x scripts/install_ollama.sh
export OLLAMA_MODEL=${OLLAMA_MODEL:-phi3}
export OLLAMA_PORT=${OLLAMA_PORT:-11434}
scripts/install_ollama.sh

[install_ollama] Downloading Ollama installer...
[install_ollama] Ollama already running. Skipping start.
[install_ollama] Pulling model phi3...
[install_ollama] Done.


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 633fc5be925f: 100% ▕██████████████████▏ 2.2 GB                         
pulling fa8235e5b48f: 100% ▕██████████████████▏ 1.1 KB                         
pulling 542b217f179c: 100% ▕██████████████████▏  148 B                         
pulling 8dde1baf1db0: 100% ▕██████████████████▏   78 B                         
pulling 23291dc44752: 100% ▕██████████████████▏  4

In [37]:
import asyncio
import os
import threading
import time

import uvicorn

import sys
sys.path.append("/content/colab_fastapi")

FASTAPI_PORT = int(os.environ['FASTAPI_PORT'])

if 'SERVER_THREAD' in globals():
    print('FastAPI server already running.')
else:
    config = uvicorn.Config('app.main:app', host='0.0.0.0', port=FASTAPI_PORT, log_level='info')
    server = uvicorn.Server(config)

    def _run_server():
        asyncio.run(server.serve())

    SERVER_THREAD = threading.Thread(target=_run_server, daemon=True)
    SERVER_THREAD.start()
    time.sleep(3)
    print(f'FastAPI server started on port {FASTAPI_PORT}.')

FastAPI server already running.


In [38]:
import os

from scripts.start_ngrok import start_ngrok

public_url = start_ngrok(port=int(os.environ['FASTAPI_PORT']), authtoken=os.environ.get('NGROK_AUTHTOKEN'))
public_url

ERROR:pyngrok.process.ngrok:t=2025-11-14T10:38:40+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-14T10:38:40+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
import asyncio
import base64
import io
import os

import cv2
import httpx
import numpy as np

FASTAPI_ROOT = f"http://127.0.0.1:{os.environ['FASTAPI_PORT']}"

async def run_smoke_tests():
    async with httpx.AsyncClient(timeout=120) as client:
        gen_payload = {
            'prompt': 'Say hello from the Colab FastAPI service.',
            'model': os.environ.get('OLLAMA_MODEL'),
        }
        gen_resp = await client.post(f'{FASTAPI_ROOT}/ollama/generate', json=gen_payload)
        gen_resp.raise_for_status()
        print('Ollama /generate →', gen_resp.json())

        dummy = np.zeros((320, 320, 3), dtype=np.uint8)
        cv2.putText(dummy, 'COLAB', (30, 170), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
        success, buffer = cv2.imencode('.jpg', dummy)
        if not success:
            raise RuntimeError('Failed to encode dummy image.')
        files = {'file': ('dummy.jpg', buffer.tobytes(), 'image/jpeg')}
        detect_resp = await client.post(f'{FASTAPI_ROOT}/yolo/detect', files=files)
        detect_resp.raise_for_status()
        print('YOLO /detect →', detect_resp.json())

asyncio.run(run_smoke_tests())

### Cleanup
Run the cell below when you need to stop the server and ngrok tunnel.

In [ ]:
import asyncio

if 'server' in globals():
    asyncio.run(server.shutdown())
    print('FastAPI server shutdown requested.')
if 'SERVER_THREAD' in globals():
    SERVER_THREAD.join(timeout=5)
    print('Server thread joined.')

from pyngrok import ngrok
ngrok.kill()
print('ngrok tunnel closed.')